# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/decl_suit-1278000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = 'KJ7432.9.A.QJT86'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "PAD_START", "PASS", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 2, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1S', 'insta_score': 0.529}, {'call': '2S', 'insta_score': 0.459}]

In [4]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.967}]

In [5]:
bid.samples

[]

In [6]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D", "PASS", "3N" , "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 0.997,
  'expected_score': -394,
  'adjustment': 498},
 {'call': 'X',
  'insta_score': 0.003,
  'expected_score': -956,
  'adjustment': -199}]

In [7]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
KJxxxx.9.A.QJT8x Q9x.KJ8xx.Kxx.9x 8.ATxx.QJ8x.AKxx ATx.Qxx.T9xxx.xx
KJxxxx.9.A.QJT8x Q9x.KQ8xx.JTx.xx 8.AJxx.9xxx.AKxx ATx.Txx.KQ8xx.9x
KJxxxx.9.A.QJT8x A98.AJx.T8xx.9xx x.KTxx.QJxxx.Axx QTx.Q8xxx.K9x.Kx
KJxxxx.9.A.QJT8x A98.J8xxx.Qxx.Kx x.AKTx.Kxxxx.xxx QTx.Qxx.JT98.A9x
KJxxxx.9.A.QJT8x A8x.Q8x.KJ8xx.9x Q.Txxx.Qxxx.AKxx T9x.AKJxx.T9x.xx
KJxxxx.9.A.QJT8x Q9x.JTxx.xxx.Axx A.AQ8x.KJTxx.9xx T8x.Kxxx.Q98x.Kx
KJxxxx.9.A.QJT8x A8x.K8xx.Qxx.9xx Q.JTxx.T8xx.AKxx T9x.AQxx.KJ9xx.x
KJxxxx.9.A.QJT8x A8x.JTxxx.Txxx.9 T.8xxx.KQJ98.Axx Q9x.AKQ.xxx.Kxxx
KJxxxx.9.A.QJT8x AQ.Qxxx.JT9x.9xx 8x.Axxx.xx.AKxxx T9x.KJT8.KQ8xxx.
KJxxxx.9.A.QJT8x Qxx.AT8x.8xx.9xx 9.QJxx.KQ9xx.Kxx AT8.Kxxx.JTxx.Ax
